In [1]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from IPython.display import display

In [2]:
# Load the dataset from csv file using pandas
data = pd.read_csv('D:\\Lomba Hackaton BCA\\fraud_detection\\data_input_csv\\data_input\\fraud_train.csv',index_col=False)
# filling missing data
data.isnull().sum()
data.fillna(0, inplace = True)

In [ ]:
data.head()

In [ ]:
data.dtypes

In [3]:
# Ubah beberapa columns integer menjadi string object
data['tipe_kartu'] = data.tipe_kartu.astype(str)
data['tipe_transaksi'] = data.tipe_transaksi.astype(str)
data['kepemilikan_kartu'] = data.kepemilikan_kartu.astype(str)

In [ ]:
data.dtypes

In [ ]:
data.columns

In [ ]:
%matplotlib inline

Sekarang, mari kita pisahkan terlebih dahulu transaksi yg fraud dengan yg non fraud

In [4]:
fraud_transaction = data.loc[data['flag_transaksi_fraud'] == 1, : ]
non_fraud = data.loc[data['flag_transaksi_fraud'] == 0, : ]

In [5]:
fraud_transaction.head()
file_name = 'fraud_transaction.csv'
fraud_transaction.to_csv(file_name, index = False, sep=',', encoding='utf-8')

In [6]:
non_fraud.head()
file_name = 'non_fraud.csv'
non_fraud.to_csv(file_name, index = False, sep=',', encoding='utf-8')

In [ ]:
fraud_transaction.tipe_transaksi.value_counts().plot(kind='bar')
plt.title('Jumlah Transaksi Fraud per Tipe Transaksi')
plt.xlabel('Tipe Transaksi')
plt.ylabel('Counts')
plt.show()

In [ ]:
non_fraud.tipe_transaksi.value_counts().plot(kind='bar')
plt.title('Jumlah Transaksi Non-Fraud per Tipe Transaksi')
plt.xlabel('Tipe Transaksi')
plt.ylabel('Counts')
plt.show()

In [ ]:
fraud_transaction.tipe_kartu.value_counts().plot(kind='bar')
plt.title('Jumlah Transaksi Fraud per Tipe Kartu')
plt.xlabel('Tipe Kartu')
plt.ylabel('Counts')
plt.show()

In [ ]:
non_fraud.tipe_kartu.value_counts().plot(kind='bar')
plt.title('Jumlah Transaksi Non-Fraud per Tipe Kartu')
plt.xlabel('Tipe Kartu')
plt.ylabel('Counts')
plt.show()

In [7]:
time = data['waktu_transaksi']
nilai_transaksi = data['nilai_transaksi']
rata_rata_nilai_transaksi = data['rata_rata_nilai_transaksi']

In [8]:
time = time.tolist()
nilai_transaksi = nilai_transaksi.tolist()
rata_rata_nilai_transaksi = rata_rata_nilai_transaksi.tolist()

In [9]:
# change_format_time berfungsi utk mengubah format time menjadi 'detik'
def change_format_time(time):
    hour = int(time / 10000)
    minute = int((time - (hour*10000))/100)
    second = time - (hour*10000) - (minute*100)
    result = (hour*3600) + (minute*60) + second
    return result

selisih_transaksi = []

# Hitung selisih
def hitung_selisih(rata_rata, nilai_transaksi):
    result = abs(rata_rata - nilai_transaksi)
    return result

# result akan menampung format data 'time' dalam detik
new_time = []

for element in time:
    result = change_format_time(element)
    new_time.append(result)

In [10]:
"""
Pembagian waktu berdasarkan tingkat aktifitas :
    Jam istirahat : 22.00 - 07.59 --> Kategori 1
    Jam kerja : 08.00 - 18.59 --> Kategori 2
    Jam santai : 19.00 - 21.59 --> Kategori 3
"""

grouped_time = []

def categorize_time(time):
    if (time >= 79200) and (time <= 86399):
        grouped_time.append(1)
    elif (time <= 28799):
        grouped_time.append(1)
    elif (time >= 28800) and (time <= 68399):
        grouped_time.append(2)
    elif (time >= 68400) and (time <= 79199):
        grouped_time.append(3)
    else:
        pass

for element in new_time:
    result = categorize_time(element)
    grouped_time.append(result)

for i in range(0, len(nilai_transaksi)):
    result = abs(nilai_transaksi[i] - rata_rata_nilai_transaksi[i])
    selisih_transaksi.append(result)

data['grouped_time'] = [x for x in grouped_time if x is not None]
data['selisih_transaksi'] = [x for x in selisih_transaksi]

grouped_time = []
selisih_transaksi = []

In [11]:
data['grouped_time'] = data.grouped_time.astype(str)

In [ ]:
data.dtypes

In [ ]:
print(data.shape)

In [ ]:
print(data.describe())

In [ ]:
# Plot histogram of each paramater
data.hist(figsize = (20, 20))
plt.show()

In [ ]:
# Determine number of fraud cases in dataset
fraud = data[data['flag_transaksi_fraud'] == 1]
valid = data[data['flag_transaksi_fraud'] == 0]

outlier_fraction = len(fraud) / float(len(valid))
print(outlier_fraction)

print('Fraud Cases: {}'.format(len(fraud)))
print('Valid Cases: {}'.format(len(valid)))

In [ ]:
# Correlation matrix
corr_mat = data.corr()
fig = plt.figure(figsize = (12, 9))

sns.heatmap(corr_mat, vmax = .8, square = True)
plt.show()

In [ ]:
data.columns

In [12]:
# Get the columns that we want from the Dataframe
columns = ['grouped_time','tipe_kartu','tipe_transaksi','selisih_transaksi','kepemilikan_kartu', 'maksimum_nilai_transaksi', 'flag_transaksi_fraud']
target = 'flag_transaksi_fraud'

In [13]:
selected_data = data.loc[:,columns]
selected_data.head()
file_name = 'data_02.csv'
selected_data.to_csv(file_name, index = False, sep=',', encoding='utf-8')

In [14]:
# over sampling
selected_data = selected_data.append([selected_data.loc[selected_data['flag_transaksi_fraud'] == 1]]*12)
#data = data.append([data]*5)
selected_data = selected_data.sample(frac = 1)

In [15]:
# check data
# Utk memastikan bahwa jumlah datanya balance
selected_data.groupby(selected_data.columns[6]).size()

flag_transaksi_fraud
0    12215
1    11830
dtype: int64

In [16]:
columns = ['grouped_time','tipe_kartu','tipe_transaksi','selisih_transaksi','kepemilikan_kartu', 'maksimum_nilai_transaksi']
X = selected_data.loc[:,columns].values
Y = selected_data.loc[:, 'flag_transaksi_fraud'].values

In [17]:
display(X)
print(type(X))
print(X.shape)

array([['1', '104', '26', 7081.443300000043, '2', 10500000.0],
       ['2', '93', '26', 2893191.192, '2', 53377977.0],
       ['1', '111', '58', 478166.73490000004, '2', 6500000.0],
       ...,
       ['2', '104', '26', 986372.1, '2', 1000000.0],
       ['2', '93', '440', 59417726.09, '1', 100000000.0],
       ['2', '1', '26', 396542.108, '2', 2500000.0]], dtype=object)

<class 'numpy.ndarray'>
(24045, 6)


In [18]:
display(Y)
print(type(Y))
print(Y.shape)

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

<class 'numpy.ndarray'>
(24045,)


In [19]:
# Encode categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
X[:, 1] = labelencoder_X.fit_transform(X[:, 1])
X[:, 2] = labelencoder_X.fit_transform(X[:, 2])
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])
# Mengubah kategorikal data menjadi dummy variable
onehotencoder = OneHotEncoder(categorical_features = [0,1,2,4]) 
X = onehotencoder.fit_transform(X).toarray()

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
# splitting datasets to Training and Testing
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Testing with Logistic Regression 

In [ ]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
############################ Evaluation ##################################
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
evaluate = precision_recall_fscore_support(y_test, y_pred)

In [ ]:
acc

In [ ]:
cm

In [ ]:
evaluate

### Testing with K-NN

In [32]:
# Fitting classifier to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [35]:
############################ Evaluation ##################################
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
evaluate = precision_recall_fscore_support(y_test, y_pred)

In [36]:
acc

0.9191100020794344

In [37]:
cm

array([[2035,  389],
       [   0, 2385]], dtype=int64)

In [38]:
evaluate

(array([1.        , 0.85976929]),
 array([0.83952145, 1.        ]),
 array([0.91276071, 0.92459779]),
 array([2424, 2385], dtype=int64))

In [26]:
X_train.shape

(19236, 41)

In [27]:
################################ ANN #####################################
from deeplab import NN

classifier = NN(3, 48, 41, 0, 1).ann()
classifier.fit(X_train, y_train, batch_size = 50, nb_epoch = 100)
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

C:\Users\sofyan.fadli\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\sofyan.fadli\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
D:\Lomba Hackaton BCA\deeplab.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=41, units=48, kernel_initializer="uniform")`
  classifier.add(Dense(output_dim = self._hidden_node, init = 'uniform', activation= 'relu', input_dim = self._input_node))
D:\Lomba Hackaton BCA\deeplab.py:29: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0)`
  classifier.add(Dropout(p = self._dropout))

Epoch 1/100
19236/19236 [==============================] - 2s 98us/step - loss: 0.5777 - acc: 0.6948
Epoch 2/100
19236/19236 [==============================] - 1s 33us/step - loss: 0.5438 - acc: 0.7295
Epoch 3/100
19236/19236 [==============================] - 1s 35us/step - loss: 0.5328 - acc: 0.7338
Epoch 4/100
19236/19236 [==============================] - 1s 32us/step - loss: 0.5232 - acc: 0.7384
Epoch 5/100
19236/19236 [==============================] - 1s 32us/step - loss: 0.5167 - acc: 0.7439
Epoch 6/100
19236/19236 [==============================] - 1s 32us/step - loss: 0.5071 - acc: 0.7476
Epoch 7/100
19236/19236 [==============================] - 1s 33us/step - loss: 0.5006 - acc: 0.7506
Epoch 8/100
19236/19236 [==============================] - 1s 32us/step - loss: 0.4934 - acc: 0.7543
Epoch 9/100
19236/19236 [==============================] - 1s 32us/step - loss: 0.4874 - acc: 0.7566
Epoch 10/100
19236/19236 [==============================] - 1s 33us/step - loss: 0.4794 - a

Epoch 81/100
19236/19236 [==============================] - 1s 35us/step - loss: 0.3208 - acc: 0.8498
Epoch 82/100
19236/19236 [==============================] - 1s 34us/step - loss: 0.3196 - acc: 0.8532
Epoch 83/100
19236/19236 [==============================] - 1s 34us/step - loss: 0.3230 - acc: 0.8497
Epoch 84/100
19236/19236 [==============================] - 1s 34us/step - loss: 0.3195 - acc: 0.8501
Epoch 85/100
19236/19236 [==============================] - 1s 37us/step - loss: 0.3197 - acc: 0.8518
Epoch 86/100
19236/19236 [==============================] - 1s 33us/step - loss: 0.3209 - acc: 0.8504
Epoch 87/100
19236/19236 [==============================] - 1s 34us/step - loss: 0.3128 - acc: 0.8556
Epoch 88/100
19236/19236 [==============================] - 1s 34us/step - loss: 0.3140 - acc: 0.8538
Epoch 89/100
19236/19236 [==============================] - 1s 32us/step - loss: 0.3108 - acc: 0.8581
Epoch 90/100
19236/19236 [==============================] - 1s 32us/step - loss: 0

In [28]:
############################ Evaluation ##################################
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
evaluate = precision_recall_fscore_support(y_test, y_pred)

In [29]:
acc

0.8440424204616345

In [30]:
cm

array([[1882,  542],
       [ 208, 2177]], dtype=int64)

In [31]:
evaluate

(array([0.90047847, 0.80066201]),
 array([0.77640264, 0.91278826]),
 array([0.83385024, 0.85305643]),
 array([2424, 2385], dtype=int64))

In [ ]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
############################ Evaluation ##################################
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
evaluate = precision_recall_fscore_support(y_test, y_pred)

In [ ]:
acc

In [ ]:
cm

In [ ]:
evaluate